# Предобработка данных

### Pip install

In [1]:
%pip install langchain-community
%pip install pypdf
%pip install langchain-ollama
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pypdf in c:\users\zenfo\appdata\local\programs\python\python312\lib\site-packages (5.1.0)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Загрузка 

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader_1 = PyPDFLoader("../data/standard_data/sql_primer.pdf")
loader_2 = PyPDFLoader("../data/standard_data/Tutorial_PostgreSQL.pdf")

### Соединение данных

In [3]:
docs = loader_1.load()
docs += loader_2.load()

print(len(docs)) 

512


In [4]:
docs

[Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 1}, page_content='Компания Postgres Professional\nЕ. П. Моргунов\nPostgreSQL. Основы языка SQL\nУчебное пособие\nСанкт-Петербург\n«БХВ-Петербург»\n2018'),
 Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 2}, page_content='УДК 004.655\nББК 32.973.26-018.2\nМ79\nМоргунов, Е. П.\nМ79 PostgreSQL. Основы языка SQL: учеб. пособие / Е. П. Моргунов; под ред.\nЕ. В. Рогова, П. В. Лузанова. — СПб.: БХВ-Петербург, 2018. — 336 с.: ил.\nISBN 978-5-9775-4022-3\nУчебно-практическое пособие охватывает первую, базовую, часть учеб-\nного курса по языку SQL, созданного при участии российской компании\nPostgres Professional. Учебный материал излагается в расчете на использо-\nвание системы управления базами данных PostgreSQL. Рассмотрено созда-\nние рабочей среды, описаны языки определения данных и осн

### Обработка данных от html и прочих символов

Обработка не доделана с ней надо подробне разбраться, смотреть как работает.
* у нас есть код, смотрите чтоб его не снесли

In [5]:
from bs4 import BeautifulSoup
import re
from langchain_core.documents import Document

# Функция для очистки текста
def clean_text(doc):
    soup = BeautifulSoup(doc, "html.parser")
    cleaned_text = soup.get_text()
    cleaned_text = re.sub(r'\n+', ' ', cleaned_text)  # Заменяем переносы строк на пробел
    cleaned_text = re.sub(r'©+', '', cleaned_text)  # Удаляем символ ©
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Удаляем повторяющиеся пробелы
    cleaned_text = re.sub(r'(\.\s*){2,}', '. ', cleaned_text) # Удаляем повторяющиеся точки с пробелом
    cleaned_text = cleaned_text = re.sub(r'- ', '', cleaned_text) # Удаляем тире с пробелом
    
    return cleaned_text

# Обработка документов
cleaned_documents = []
for doc in docs:
    cleaned_content = clean_text(doc.page_content)
    cleaned_documents.append(Document(metadata=doc.metadata, page_content=cleaned_content))



In [6]:
cleaned_documents

[Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 1}, page_content='Компания Postgres Professional Е. П. Моргунов PostgreSQL. Основы языка SQL Учебное пособие Санкт-Петербург «БХВ-Петербург» 2018'),
 Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 2}, page_content='УДК 004.655 ББК 32.973.26-018.2 М79 Моргунов, Е. П. М79 PostgreSQL. Основы языка SQL: учеб. пособие / Е. П. Моргунов; под ред. Е. В. Рогова, П. В. Лузанова. — СПб.: БХВ-Петербург, 2018. — 336 с.: ил. ISBN 978-5-9775-4022-3 Учебно-практическое пособие охватывает первую, базовую, часть учебного курса по языку SQL, созданного при участии российской компании Postgres Professional. Учебный материал излагается в расчете на использование системы управления базами данных PostgreSQL. Рассмотрено создание рабочей среды, описаны языки определения данных и основные операции выборки 

### Перевод в формат для векторизации

Перевод в другой формат для векторизации.
Вообще скорее всего переводить было не обязательно, достаточно найти нормальный метод из предложенных разделения на чанки text_splitter  
https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/

тут возникает проблема
1) если не переводить в формат Document то есть проблемы с чанками которую можно решить поменяв разделитель, читайте инф по ссылке
2) есили всё же переводить что я делаю, появляюся лишние символы, множество точек, из - за чего чанки становятся бессмысленными просмотрите
    * cleaned_content - плохо но прдобработаны
    * documents - получившийся из них с точками /н и подобной ересью 

нужно разобраться, как разбить на чанки не потеряв обработку данных на чистоту

### Разбиение на чанки

In [7]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator=". ",
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

chunked_documents = []

# Разделение текста на чанки
for doc in cleaned_documents:
    if doc.page_content:  # Проверяем, что page_content не пустой
        chunks = text_splitter.split_text(doc.page_content)
        
        # Создаем новый Document для каждого чанка
        for chunk in chunks:
            chunked_doc = Document(
                metadata=doc.metadata,  # Сохраняем метаданные
                page_content=chunk      # Сохраняем чанк текста
            )
            chunked_documents.append(chunked_doc)

Created a chunk of size 305, which is longer than the specified 300
Created a chunk of size 321, which is longer than the specified 300
Created a chunk of size 365, which is longer than the specified 300
Created a chunk of size 493, which is longer than the specified 300
Created a chunk of size 308, which is longer than the specified 300
Created a chunk of size 318, which is longer than the specified 300
Created a chunk of size 590, which is longer than the specified 300
Created a chunk of size 321, which is longer than the specified 300
Created a chunk of size 350, which is longer than the specified 300
Created a chunk of size 400, which is longer than the specified 300
Created a chunk of size 385, which is longer than the specified 300
Created a chunk of size 349, which is longer than the specified 300
Created a chunk of size 429, which is longer than the specified 300
Created a chunk of size 484, which is longer than the specified 300
Created a chunk of size 524, which is longer tha

In [8]:
chunked_documents

[Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 1}, page_content='Компания Postgres Professional Е. П. Моргунов PostgreSQL. Основы языка SQL Учебное пособие Санкт-Петербург «БХВ-Петербург» 2018'),
 Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 2}, page_content='УДК 004.655 ББК 32.973.26-018.2 М79 Моргунов, Е. П. М79 PostgreSQL. Основы языка SQL: учеб. пособие / Е. П. Моргунов; под ред. Е. В. Рогова, П. В. Лузанова. — СПб.: БХВ-Петербург, 2018. — 336 с.: ил'),
 Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 2}, page_content='— СПб.: БХВ-Петербург, 2018. — 336 с.: ил. ISBN 978-5-9775-4022-3 Учебно-практическое пособие охватывает первую, базовую, часть учебного курса по языку SQL, созданного при участии российской компании Postgres Professional'),
 Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 2}, page_content='Учебный материал излагается в расчете на использование сис

нужно поэксперементировать с данными и просмотреть чанки, на данный момент они несут практически 0 смысл


### Векторизация

требует больших мощностей, нужно запускать с мощьного пк, посмотреть возможность работы на видюхе, поспрашивайте гпт

In [9]:
# создаём путь до места хранения
from pathlib import Path

# Получаем текущий рабочий каталог
current_dir = Path.cwd()

# Создаем путь к папке processed_data, которая находится на уровне выше текущего каталога
processed_data_path = current_dir.parent / 'data' / 'processed_data' / 'union.parquet'


In [13]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import OllamaEmbeddings

# Create embeddings for documents and store them in a vector store
vectorstore = SKLearnVectorStore.from_documents(
    documents=chunked_documents[:40],
    embedding = OllamaEmbeddings(model="llama3.1"),
    persist_path= processed_data_path,  # for save
    serializer="parquet" # for save
)

### Сохранение

In [14]:
# save
vectorstore.persist()

In [15]:
chunked_documents[:40]

[Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 1}, page_content='Компания Postgres Professional Е. П. Моргунов PostgreSQL. Основы языка SQL Учебное пособие Санкт-Петербург «БХВ-Петербург» 2018'),
 Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 2}, page_content='УДК 004.655 ББК 32.973.26-018.2 М79 Моргунов, Е. П. М79 PostgreSQL. Основы языка SQL: учеб. пособие / Е. П. Моргунов; под ред. Е. В. Рогова, П. В. Лузанова. — СПб.: БХВ-Петербург, 2018. — 336 с.: ил'),
 Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 2}, page_content='— СПб.: БХВ-Петербург, 2018. — 336 с.: ил. ISBN 978-5-9775-4022-3 Учебно-практическое пособие охватывает первую, базовую, часть учебного курса по языку SQL, созданного при участии российской компании Postgres Professional'),
 Document(metadata={'source': '../data/standard_data/sql_primer.pdf', 'page': 2}, page_content='Учебный материал излагается в расчете на использование сис